In [1]:
import os

In [2]:
%pwd

'/home/nikhilds/Projects/Kidney-Disease-Classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/nikhilds/Projects/Kidney-Disease-Classification'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_include_top: bool
    params_classes: int
    params_learning_rate: float
    params_weights: str

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_path=CONFIG_FILE_PATH,
            params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self)->PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_learning_rate=self.params.LEARNING_RATE
        )

        return prepare_base_model_config

In [8]:
import os
import tensorflow as tf
from zipfile import ZipFile
import urllib.request as request

2024-09-30 22:26:12.172895: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-30 22:26:12.383520: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-30 22:26:12.385043: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path,model=self.model)
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

    @staticmethod
    def prepare_base_model(model,freeze_all,freeze_till,classes,learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif(freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_layer = tf.keras.layers.Flatten()
        flatten_out = flatten_layer(model.output)
        
        fc1 = tf.keras.layers.Dense(
            units=128,
            activation='relu'
        )(flatten_out)
        fc2 = tf.keras.layers.Dense(
            units=64,
            activation='relu'
        )(fc1)
        fc3 = tf.keras.layers.Dense(
            units=32,
            activation='relu'
        )(fc2)
        predictions = tf.keras.layers.Dense(
            units=classes,
            activation='sigmoid'
        )(fc3)

        final_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = predictions
        )

        final_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss = 'binary_crossentropy',
            metrics = ['accuracy']
        )
        final_model.summary()
        return final_model
    
    def get_updated_model(self):
        self.full_model = self.prepare_base_model(
            model = self.model,
            freeze_all = True,
            freeze_till = None,
            classes = self.config.params_classes,
            learning_rate = self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)

In [14]:
try:
    config = ConfigurationManager()
    base_model_config = config.get_prepare_base_model_config()
    base_model = PrepareBaseModel(config=base_model_config)
    base_model.get_base_model()
    base_model.get_updated_model()
except Exception as e:
    raise e

[2024-09-30 23:45:54,737: INFO: common: yaml file: config/config.yaml loaded successfully:]
[2024-09-30 23:45:54,739: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-09-30 23:45:54,740: INFO: common: created directory at: artifacts:]
[2024-09-30 23:45:54,740: INFO: common: created directory at: artifacts/prepare_base_model:]
[2024-09-30 23:45:54,986: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.:]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 22